# convertFIREData_custom.ipynb

This is an advanced tutorial using FIREreader, be warned!!

This notebook can be used on Stampede2, where the halo file and snapshot directories live. You can run this notebook, and host a Firefly server, on Stampede by following the instructions [here](https://github.com/ageller/Firefly/wiki/Hosting-Firefly-on-a-Cluster-Environment). 

In this notebook, we open the AHF halo files saved on Stampede and offset the snapshot coordinates, as well as convert them to physical units, to put the center of the main halo at our origin. This is optional, since you can always fly within Firefly to a point and set that as your origin, but more convenient (and exact!). 

We also calculate the radius from the halo center for each particle and update the filter keys so we can interactively filter by radius from within Firefly. 

In [1]:
%load_ext autoreload
%autoreload 2

from dataParser import FIREreader
import numpy as np
import os
import h5py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize the Reader instance
We'll ask for gas and star particles, along with their density, velocity, temperature, and age in gyr as appropriate. The latter two are enabled by Alex Gurvich's openSnapshot utility. You're welcome!

Note that a quirk of how Velocity filtering is done (the magnitude of velocity, a vector quantity vs. the quantity itself) we have to pass filterFlag = 0 for Velocity (note also that we never use doMag = 1). This will likely change in the future. 

In [2]:
## initialize reader object and choose simulation to run
reader = FIREreader(
    snapdir = "/Users/agurvich/research/snaps/m12i_res7100/output",
    snapnum = 600,
    JSONdir='m12i_res7100_600',
    ptypes = ['PartType0','PartType4'],
    UInames = ['Gas','Stars'],
    dec_factors = [100,100],
    returnKeys = ['Density','Velocities','Temperature','AgeGyr'], ## keys that don't match a particle type are ignored
    filterFlags = [1,0,1,1],
    colormapFlags = [1,0,1,1],
    doMags = [0,0,0,0],
    doLogs = [1,0,1,0],
    )
## could read this from snapshot times
current_redshift=0

### Load the data into the reader

In [3]:
reader.loadData()

Loading ptype PartType0
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
This is a cosmological snapshot... converting to physical units
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.1.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.2.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.3.hdf5
Loading ptype PartType4
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
This is a cosmological snapshot... converting to physical units
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.1.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.2.hdf5
/Users/agurvich/research/snaps/m12i_res7100/output/snapdir_600/snapshot_600.3.hdf5


[Particle Group of Gas
 Contains 56897231 (568972 after dec) particles and 3 arrays,
 Particle Group of Stars
 Contains 13728304 (137283 after dec) particles and 2 arrays]

## Open the AHF Halo file and extract the halo center

In [4]:
def load_AHF(directory,snapnum,current_redshift,hubble = 0.702):
        path = os.path.join(directory,'../halo/ahf/halo_00000_smooth.dat')
        
        ## find column numbers without having to count
        names_to_read = ['snum','Xc','Yc','Zc','Rvir','v_esc','Rstar0.5']
        
        ## load the first line of the datafile
        names = list(np.genfromtxt(path,skip_header=0,max_rows = 1,dtype=str))
        cols = []

        ## find the column each name appears in
        for name in names_to_read:
            cols+=[names.index(name)]

        ## load the rest of the file
        sns,xs,ys,zs, rvirs, vescs, rstar_halfs = np.genfromtxt(
            path,delimiter='\t',usecols=cols,unpack=1,skip_header=1)

        ## which row do I care about? make an index array
        index = sns==snapnum
        if np.sum(index)==0:
            ## snapnum is not in this halo file
            raise IOError
            
        ## presumably in comoving kpc/h 
        halo_center = np.array([xs[index],ys[index],zs[index]])/hubble*(1/(1+current_redshift))
        halo_center = halo_center.reshape(3,)

        ## convert other quantities one might care about from comoving kpc to pkpc
        rvir = rvirs[index][0]/hubble/(1+current_redshift)
        vesc = vescs[index][0]
        rstar_half = rstar_halfs[index][0]/hubble/(1+current_redshift)
        return halo_center, rvir, vesc, rstar_half

In [5]:
## open the halo file and find the center
halo_center,rvir,vesc,rstar_half = load_AHF(reader.snapdir,reader.snapnum,current_redshift)
print halo_center,rvir

[ 41875.75818899  44122.37307211  46257.47577379] 273.803418803


## Fine tune the particleGroups
We can change the default color of each particle group and adjust the units/offsets for various quantities. For instance, here, we'll 
* change the density into physical units
* calculate the galactocentric radius and make them filterable
* change the default color
* change the default size for both particle groups
* offset the coordinates by the halo's center of mass so that we can center our view. 

In [6]:
colors = [
    [1,0,0,1],## gas is red
    [0,0,1,1] ## stars are blue
]

# Code mass -> g , (code length)^-3 -> cm^-3 , g -> nHydrogen
DENSITYFACT=2e43*(3.086e21)**-3/(1.67e-24)

for i,particleGroup in enumerate(reader.particleGroups):
    particleGroup.coordinates-=halo_center ## both are already in physical units

    ## now calculate the galactocentric radii and track them
    radii = np.sum(particleGroup.coordinates**2,axis=1)**0.5
    particleGroup.trackArray('Radius',radii,filter_flag=1)

    ## let's convert the density into physical units
    if 'log10Density' in particleGroup.tracked_names:
        ## gas is the first particle group
        particleGroup['log10Density'] = particleGroup['log10Density']+np.log10(DENSITYFACT)

    ## change some of the options like the size of the particles and their color
    reader.options['sizeMult'][particleGroup.UIname]=0.3 ## set the particle size
    reader.options['color'][particleGroup.UIname]=colors[i] ## set the particle colors


## Reopening the Snapshot File
You might have to reopen the snapshot files to get at arrays that weren't opened in returnKeys (let's say because you didn't want to keep track of a mass array and just inflate the size of your `.json` files). You can do this by using the `particleGroup.filenames_opened` attribute that the `FIREreader` class tracks for each particleGroup. 

As an example of this, let's remove the center of mass velocity from both particle group's velocities so that velocity vectors are accurate. To do that we will need to 
* Open each of the snapshots and extract the masses
* find the stars near to the main halo
* compute the center of mass velocity
* subtract the center of mass velocity from each of the `Velocities` `tracked_array`s


In [7]:
assert particleGroup.UIname == 'Stars'
star_masses = []
## open each of the files and extract the masses
for filename in particleGroup.filenames_opened:
    with h5py.File(filename,'r') as handle:
        star_masses = np.append(star_masses,handle['PartType4/Masses'])

In [8]:
## find the nearby star particles
near_star_indices = particleGroup['Radius'] < 50 ## kpc
## calculate vcom
near_star_vcom = (
    np.sum(star_masses[near_star_indices][:,None]
    *particleGroup['Velocities'][near_star_indices],axis=0)
    /np.sum(star_masses[near_star_indices])
)

print(near_star_vcom,'kms')

## now let's remove it from the particle velocities
for particleGroup in reader.particleGroups:
    particleGroup['Velocities']-=near_star_vcom

(array([-49.62116791,  75.18697612,  95.4050726 ]), 'kms')


## Output our Reader instance's info to JSON

In [9]:
reader.dumpToJSON()

outputting Particle Group of Gas
Contains 56897231 (568972 after dec) particles and 4 arrays

['log10Density', 'Velocities', 'log10Temperature', 'Radius'] [1, 0, 1, 1]
outputting Particle Group of Stars
Contains 13728304 (137283 after dec) particles and 3 arrays

['Velocities', 'AgeGyr', 'Radius'] [0, 1, 1]
